In [ ]:
!pip install azure-storage-blob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.2/392.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.9 MB/s eta 0:00:00


In [ ]:
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 55.7 MB/s eta 0:00:00


In [ ]:
from azure.storage.blob import BlobServiceClient, ContainerClient, BlobClient

import mysql.connector as msql
from mysql.connector import Error, connect

from getpass import getpass

import pandas as pd

String de conexão com o Storage Account na nuvem Azure

In [ ]:
# Configurações de conexão
account_name = 'avgussto'
account_key = 'exemplo'

# Criando o cliente de serviço de blob
blob_service_client = BlobServiceClient(
    account_url=f"https://{account_name}.blob.core.windows.net",
    credential=account_key
)
# Nome do contêiner que você deseja acessar
container_name = 'checkpint-cloud'

# Obtendo o cliente de contêiner
container_client = blob_service_client.get_container_client(container_name)

# Agora você pode usar o objeto 'container_client' para interagir com o contêiner
# Por exemplo, listar blobs no contêiner
blob_list = container_client.list_blobs()
for blob in blob_list:
    print(blob.name)


base_transacional_supermercado_hash.csv


In [ ]:
# Baixar o arquivo CSV do armazenamento
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob.name)

with open("base_transacional_supermercado_hash.csv", "wb") as meu_csv:
    blob_data = blob_client.download_blob()
    meu_csv.write(blob_data.readall())

In [ ]:
# Ler o arquivo CSV com o Pandas
v_dados = pd.read_csv(meu_csv.name, encoding="latin1")

# Agora você pode trabalhar com o DataFrame normalmente
v_dados.columnss

Index(['Unnamed: 0', 'CodFamilia', 'PRODUCT_ID', 'STORE_ID', 'AMOUNT', 'DATE',
       'DISCOUNT_AMOUNT', 'DISPLAY_FLAG', 'FEATURE_FLAG',
       'GROSS_PROFIT_AMOUNT', 'POINTS_BONUS_AMOUNT', 'QUANTITY', 'CATEGORY_1',
       'CATEGORY_2', 'CATEGORY_3', 'CATEGORY_4', 'NAME', 'hash'],
      dtype='object')

In [ ]:
v_dados.head()

,Unnamed: 0,CodFamilia,PRODUCT_ID,STORE_ID,AMOUNT,DATE,DISCOUNT_AMOUNT,DISPLAY_FLAG,FEATURE_FLAG,GROSS_PROFIT_AMOUNT,POINTS_BONUS_AMOUNT,QUANTITY,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,NAME,hash
0,0,138104,196282,409,8.98,2020-01-01 00:14:41 UTC,0.0,0,0,3.875962,40,2.0,MERCEARIA,BOMBONIERE SECAO,BOMBONS-CHOCOLATES,CHOC BARRA-TABLETE,CHOC GAROTO 90G-TA CAST CAJU,1835784604519550211
1,1,97760,136956,62,0.00,2020-01-01 00:10:16 UTC,0.0,0,0,-60.099320,120,4.0,MERCEARIA,BEBIDAS,VINHOS,VIN CHILENO,VIN CHIL ANTAKARI RSV 750ML CHARD,-1083971162102878296
2,2,247,329,408,9.98,2020-01-01 00:08:29 UTC,0.0,0,0,2.965819,20,1.0,MERCEARIA,MATINAIS SECAO,CAFES,CAFE CAPPUCINO-COM LEITE,CAPPUCCINO 3CORACOES CLAS 200G-PT,1440480164372428991
3,3,77004,104089,405,7.99,2020-01-01 00:06:10 UTC,0.0,0,0,1.807759,20,1.0,MERCEARIA,CONSERVAS-CONDIMENTOS,CONDIMENTOS-TEMPEROS,CATCHUP FR ATE 500G,KETCHUP HEINZ 397G-PET TRAD,-6447732333630099708
4,4,138045,196180,62,11.96,2020-01-01 00:05:30 UTC,0.0,0,0,3.739260,30,2.0,PERECIVEIS,P.A.S.,VEGETAIS CONG,LEGUME CONG,ANEIS CEBOLA GOLDEN FOODS 350G-PC,-3203853051821202807


In [ ]:
v_dados['DATE'] = pd.to_datetime(v_dados['DATE'])

In [ ]:
# Servidor 01

v_host = "168.138.147.231"

v_user = input("Usuário: ")
v_passwd = getpass("Senha: ")

v_database = input("Database: ")

try:
    cnx = msql.connect(user=v_user, password=v_passwd, database=v_database, host=v_host, auth_plugin="caching_sha2_password")

    if cnx.is_connected():
      v_cursor = cnx.cursor()
      v_cursor.execute("SELECT DATABASE()")
      v_registro = v_cursor.fetchone()

      v_cursor.execute("SET autocommit=0")

      print("Conectado na base de dados: ", v_registro)

    if cnx.is_connected():
        v_cursor = cnx.cursor()

        v_count = 0;
        v_commit = 1000;

        for i, row in v_dados.iterrows():
            sql = "INSERT INTO SUPERMERCADO VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

            v_cursor.execute(sql, tuple(row))

            v_count = i;

            if (v_count % v_commit) == 0:
                print("Registro(s) Inserido(s): ", v_count)
                cnx.commit()
        print("Registro(s) Inserido(s): ", v_count)
        cnx.commit()

        v_cursor.close()
        cnx.close()


except Error as e:
    print("Erro na conexão com MySQL", e)

Conectado na base de dados:  ('DB_RM551331',)
Registro(s) Inserido(s):  0
Registro(s) Inserido(s):  1000
Registro(s) Inserido(s):  2000
Registro(s) Inserido(s):  3000
Registro(s) Inserido(s):  4000
Registro(s) Inserido(s):  5000
Registro(s) Inserido(s):  6000
Registro(s) Inserido(s):  7000
Registro(s) Inserido(s):  8000
Registro(s) Inserido(s):  9000
Registro(s) Inserido(s):  10000
Registro(s) Inserido(s):  11000
Registro(s) Inserido(s):  12000
Registro(s) Inserido(s):  13000
Registro(s) Inserido(s):  14000
Registro(s) Inserido(s):  15000
Registro(s) Inserido(s):  16000
Registro(s) Inserido(s):  17000
Registro(s) Inserido(s):  18000


KeyboardInterrupt: 